# Walkthrough Challenge 5 - Multi-Agent Chatroom

Duration: 30 minutes

## Overview
- In this challenge, you will be introduced to the concept of agents and multi-agent collaboration.
- You will create a multi-agent chatroom where agents can communicate with each other, to solve a common task.

## Prerequisites

- Please ensure that you have completed the [Setup](../setup/setup.ipynb) before starting this challenge.

### Task 1: Configure and Initialize Semantic Kernel

⚠️ Note: You should have already completed all tasks on the [Setup](../setup/setup.ipynb). If you have not, please go back and complete it now.

#### Step 1: Load Semantic Kernel settings

In this step, we will load the Semantic Kernel settings that we created in the [Setup](../setup/setup.ipynb) notebook.

In [4]:
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.13.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.13.0-alpha"

#!import ../setup/config/Settings.cs
#!import ../setup/config/Utils.cs

Installed Packages Microsoft.SemanticKernel, 1.13.0 Microsoft.SemanticKernel.Agents.Abstractions, 1.13.0-alpha Microsoft.SemanticKernel.Agents.Core, 1.13.0-alpha

#### Step 2: Initialize Semantic Kernel

In [5]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TextToImage;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.Agents.Chat;
using System.Threading;
using System.Threading.Tasks;
using System.Collections.Generic;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("../setup/config/settings.json");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

## Task 2: Build the agent personas

In this task, we will define the personas of the agents involved in the multi-agent chatroom. Each agent has a specific role and responsibilities in the chatroom. The personas help us understand the perspective and expertise of each agent.

Agent personas are defined to establish clear roles and responsibilities within the multi-agent chatroom, enabling effective collaboration and problem-solving.

We will define the following agent personas: Product Owner, Technical Lead, UX Designer, QA Analyst and Business Analyst.

In [6]:
private const string ProductOwnerName = "ProductOwner";
private const string ProductOwnerInstructions =
    """
    You are a Product Owner responsible for ensuring the acceptance criteria align with the product vision and meet the stakeholders' requirements.
    Your goal is to initiate and validate acceptance criteria that clearly define the conditions for a user story to be considered complete.
    Consider feedback from the Technical Lead, UX Designer, and QA Analyst.
    Make the final decision on whether the acceptance criteria are ready and comprehensive.
    Provide feedback if the acceptance criteria are incomplete or unclear, suggesting improvements where necessary.
    If satisfactory, state that the acceptance criteria are approved. If they are approved, also provide the last version of the acceptance criteria.
    Be concise and to the point.
    """;

private const string TechnicalLeadName = "TechnicalLead";
private const string TechnicalLeadInstructions =
    """
    You are a Technical Lead with expertise in assessing the technical feasibility and implementation details of acceptance criteria.
    Your goal is to review the acceptance criteria for technical completeness, potential challenges, and feasibility within the current architecture.
    Provide feedback on potential technical risks or areas needing clarification, and suggest any improvements to the acceptance criteria.
    Be concise and to the point.
    Only provide a single proposal per response.
    """;

private const string UXDesignerName = "UXDesigner";
private const string UXDesignerInstructions =
    """
    You are a UX Designer focused on ensuring that the acceptance criteria support a positive user experience and adhere to design principles.
    Your goal is to review the acceptance criteria to ensure they account for user interaction, usability, and accessibility.
    Provide feedback on how the acceptance criteria can be improved to enhance the user experience.
    Be concise and to the point.
    Only provide a single proposal per response.
    """;

private const string QAAnalystName = "QAAnalyst";
private const string QAAnalystInstructions =
    """
    You are a QA Analyst responsible for ensuring that the acceptance criteria are clear, detailed, and testable.
    Your goal is to review the acceptance criteria to ensure they are clear, concise, and include specific details that can be used for testing.
    Provide feedback on any missing or unclear acceptance criteria and suggest improvements to make them more testable.
    Be concise and to the point.
    Only provide a single proposal per response.
    """;

private const string BusinessAnalystName = "BusinessAnalyst";
private const string BusinessAnalystInstructions =
    """
    You are a Business Analyst responsible for writing clear, concise, and complete acceptance criteria based on the user story.
    Your goal is to draft acceptance criteria that accurately capture the user needs, business value, and testable conditions.
    Consider feedback and suggestions from the Product Owner, Technical Lead, UX Designer, and QA Analyst to refine and improve the acceptance criteria.
    Ensure the acceptance criteria are well-structured and ready for development.
    Be concise and to the point.
    """;

## Task 3: Spawn agents with their different roles

To facilitate multi-agent collaboration, we will spawn multiple agents in the chatroom. Each agent will have a specific role and responsibilities. Here are the agents involved:

1. **Product Owner**
    - Name: ProductOwner
    - Instructions: Ensure acceptance criteria align with the product vision and stakeholder requirements.

2. **Technical Lead**
    - Name: TechnicalLead
    - Instructions: Assess technical feasibility and provide feedback on implementation details.

3. **UX Designer**
    - Name: UXDesigner
    - Instructions: Review acceptance criteria for user experience and design principles.

4. **QA Analyst**
    - Name: QAAnalyst
    - Instructions: Ensure acceptance criteria are clear, detailed, and testable.

5. **Business Analyst**
    - Name: BusinessAnalyst
    - Instructions: Write clear, concise, and complete acceptance criteria based on user needs.

By spawning these agents in the chatroom, we can leverage their expertise and collaborate effectively to solve the given task.
```

In [7]:
#pragma warning disable SKEXP0110

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentProductOwner =
    new()
    {
        Instructions = ProductOwnerInstructions,
        Name = ProductOwnerName,
        Kernel = kernel,
    };

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentTechnicalLead =
    new()
    {
        Instructions = TechnicalLeadInstructions,
        Name = TechnicalLeadName,
        Kernel = kernel,
    };

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentUXDesigner =
    new()
    {
        Instructions = UXDesignerInstructions,
        Name = UXDesignerName,
        Kernel = kernel,
    };

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentQAAnalyst =
    new()
    {
        Instructions = QAAnalystInstructions,
        Name = QAAnalystName,
        Kernel = kernel,
    };

Microsoft.SemanticKernel.Agents.ChatCompletionAgent agentBusinessAnalyst =
    new()
    {
        Instructions = BusinessAnalystInstructions,
        Name = BusinessAnalystName,
        Kernel = kernel,
    };

## Task 4: Create a termination condition

In our solution, we need to establish a termination condition for our process. This is a condition that, when met, will stop the process from continuing. In our case, we want the process to stop when the copy has been approved.

To do this, we use a KernelFunction object. This object is created using the KernelFunctionFactory.CreateFromPrompt method, which takes a string prompt as an argument. The prompt is a set of instructions that the KernelFunction will follow.

In [8]:
KernelFunction terminationFunction =
    KernelFunctionFactory.CreateFromPrompt(
        """
        Determine if the copy has been approved.  If so, respond with a single word: yes

        History:
        {{$history}}
        """);

## Task 5: Glue it all together and run the group chat

We will bring together all the components we have built so far. We will create a group chat environment where multiple agents can interact with each other, share information, and collaborate to solve a common task.

In the end, with the help of the agents, we will be able to create a set of acceptance criteria for a given user story. The agents will collaborate, share their expertise, and work together to achieve the best possible acceptance criteria, based on the user story and acceptance criteria provided by the user.

In [12]:
#pragma warning disable SKEXP0110, SKEXP0001

var chat =
    new AgentGroupChat(agentBusinessAnalyst, agentTechnicalLead, agentUXDesigner, agentQAAnalyst, agentProductOwner)
    {
        ExecutionSettings =
            new()
            {
                // Here a TerminationStrategy subclass is used that will terminate when
                // an assistant message contains the term "approved".
                TerminationStrategy =
                    new KernelFunctionTerminationStrategy(terminationFunction, kernel)
                    {
                        // Only the Product Owner may approve.
                        Agents = [agentProductOwner],
                        // Customer result parser to determine if the response is "yes"
                        ResultParser = (result) => result.GetValue<string>()?.Contains("yes", StringComparison.OrdinalIgnoreCase) ?? false,
                        // The prompt variable name for the history argument.
                        HistoryVariableName = "history",
                        // Limit total number of turns
                        MaximumIterations = 10
                    }
            }
    };

// Invoke chat and display messages.
string input =
"""
**User Story**:
As a regular user, I want to securely log into my account using my username and password, so that I can access personalized features and maintain my privacy.

**Acceptance Criteria**:
1. Users can click a "Login" button on the homepage which directs them to the login page.
2. Users enter their username and password in the provided fields.
3. A 'Login' button submits the credentials, which should be encrypted during transmission.
4. Users are redirected to their personalized dashboard upon successful authentication.
5. An error message, "Incorrect username or password. Please try again," appears for invalid credentials.
6. After three consecutive failed login attempts, the account is locked for 15 minutes to prevent unauthorized access.
7. A 'Forgot Password?' link is available that directs users to a secure password reset page.
8. The login process (from clicking 'Login' to reaching the dashboard) is achieved within 5 seconds under standard conditions.
""";

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
Console.WriteLine($"# {AuthorRole.User}: '{input}'");

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}

Console.WriteLine($"# IS COMPLETE: {chat.IsComplete}");

# user: '**User Story**:
As a regular user, I want to securely log into my account using my username and password, so that I can access personalized features and maintain my privacy.'
# assistant - BusinessAnalyst: '**Acceptance Criteria**:
1. **User Interface Display**:
   - The login page should display fields for entering a username and password.
   - The login page must include a "Submit" button to initiate the login process.

2. **Input Validation**:
   - The system should validate that both the username and password fields are not empty before submission.
   - If either field is empty upon submission attempt, the system should display an error message "Both username and password are required."
   - The password field should mask input characters for security.

3. **Authentication Process**:
   - Upon submission, the system should verify the username and password against stored credentials.
   - If the credentials are incorrect, the system should display an error message "Invalid 

Error: Command cancelled.

You successfully completed challenge 5! 🚀🚀🚀

 **[Home](../../Readme.md)**